In [ ]:
import pandas as pd
import numpy as np

# Chargement des données (remplacer 'path_to_file.csv' par le chemin vers le fichier complet)
df = pd.read_csv("ecommerce_data.csv")

# Affichage des premières lignes
print("Aperçu du dataset :")
print(df.head())

# Aperçu des informations du dataset
print("\nInformations sur le dataset :")
print(df.info())

# Statistiques descriptives des colonnes numériques
print("\nStatistiques descriptives :")
print(df.describe())

In [ ]:
# Vérification des valeurs manquantes dans chaque colonne
print("\nValeurs manquantes par colonne :")
print(df.isnull().sum())


In [ ]:
# Suppression des lignes avec des valeurs nulles dans 'customer_id' et 'customer_name'
df_cleaned = df.dropna(subset=['customer_id', 'customer_name'])

# Créer une copie du DataFrame nettoyé
df_cleaned = df_cleaned.copy()


# Remplacement des valeurs manquantes pour simplifier l'analyse
df_cleaned.fillna({
    'customer_age': df['customer_age'].mean(),
    'purchase_amount': df['purchase_amount'].mean(),
    'product_rating': df['product_rating'].mode()[0],
    'loyalty_status': 'Unknown',
    'product_category': 'Unknown',
    'delivery_status': 'Unknown',
    'payment_status': 'Unknown',
    'customer_gender': 'Unknown',
    'country': 'Unknown',
    'shipping_region': 'Unknown'
}, inplace=True)


# Transformation des colonnes catégoriques en numériques

df_cleaned['loyalty_status'] = df_cleaned['loyalty_status'].astype('category').cat.codes
df_cleaned['product_category'] = df_cleaned['product_category'].astype('category').cat.codes
df_cleaned['delivery_status'] = df_cleaned['delivery_status'].astype('category').cat.codes
df_cleaned['payment_status'] = df_cleaned['payment_status'].astype('category').cat.codes
df_cleaned['customer_gender'] = df_cleaned['customer_gender'].astype('category').cat.codes
df_cleaned['country'] = df_cleaned['country'].astype('category').cat.codes
df_cleaned['shipping_region'] = df_cleaned['shipping_region'].astype('category').cat.codes


# Vérification des valeurs manquantes après nettoyage
print("\nValeurs manquantes après nettoyage :")
print(df_cleaned.isnull().sum())


In [ ]:
from sklearn.preprocessing import StandardScaler

# Sélectionner les colonnes numériques à standardiser
columns_to_standardize = ['purchase_amount', 'customer_age', 'product_rating']
# Initialiser le StandardScaler
scaler = StandardScaler()

# Appliquer la standardisation sur les colonnes sélectionnées
df_cleaned[columns_to_standardize] = scaler.fit_transform(df_cleaned[columns_to_standardize])

# Vérification des valeurs manquantes après standardisation
print("\nValeurs manquantes après standardisation :")
print(df_cleaned.isnull().sum())

# Aperçu des premières lignes après standardisation
print("\nAperçu du dataset après standardisation :")
print(df_cleaned.head())

# Si tu veux vérifier les statistiques après standardisation :
print("\nStatistiques descriptives après standardisation :")
print(df_cleaned[columns_to_standardize].describe())


In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

# Appliquer K-Means avec un nombre de clusters choisi (par exemple, 4)
kmeans = KMeans(n_clusters=4, random_state=42)

# Appliquer le modèle KMeans sur les données standardisées
df_cleaned['cluster'] = kmeans.fit_predict(df_cleaned[columns_to_standardize])

# Afficher les centres des clusters
centers = pd.DataFrame(kmeans.cluster_centers_, columns=columns_to_standardize)
print("\nCentres des clusters :")
print(centers)

# Visualisation des clusters avec un graphique simple (exemple avec 'purchase_amount' vs 'customer_age')
sns.scatterplot(x=df_cleaned['customer_age'], y=df_cleaned['purchase_amount'], hue=df_cleaned['cluster'], palette='viridis')
plt.title("Segmentation des clients avec K-Means")
plt.xlabel("Âge du client")
plt.ylabel("Montant des achats")
plt.legend(title='Cluster')
plt.show()



In [ ]:
# Pairplot pour visualiser les relations entre plusieurs variables dans les clusters
sns.pairplot(df_cleaned, hue="cluster", vars=['purchase_amount', 'customer_age', 'product_rating', 'loyalty_status'], palette='viridis')
plt.suptitle("Segmentation des clients par K-Means", y=1.02)
plt.show()


In [ ]:
from sklearn.decomposition import PCA

# Appliquer PCA pour réduire la dimensionnalité à 2 dimensions
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_cleaned[columns_to_standardize])

# Ajouter les résultats PCA dans le DataFrame
df_cleaned['pca1'] = pca_components[:, 0]
df_cleaned['pca2'] = pca_components[:, 1]

# Visualisation des clusters avec les deux premières composantes principales
sns.scatterplot(x='pca1', y='pca2', hue='cluster', data=df_cleaned, palette='viridis')
plt.title("Segmentation des clients avec PCA et K-Means")
plt.legend(title='Cluster')
plt.show()
